<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtAlbert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 890kB 15.9MB/s 
     |████████████████████████████████| 3.0MB 21.3MB/s 
     |████████████████████████████████| 1.1MB 25.4MB/s 


In [2]:
import tensorflow as tf
from transformers import TFAlbertModel, AlbertTokenizer,AlbertConfig
import re
import numpy as np

<b>2. Load the vocabulary and ProtAlbert Model</b>

In [3]:
tokenizer = AlbertTokenizer.from_pretrained("Rostlab/prot_albert", do_lower_case=False )

In [4]:
model = TFAlbertModel.from_pretrained("Rostlab/prot_albert", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFAlbertModel

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [5]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [6]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [7]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [8]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [9]:
embedding = model(input_ids)[0]

In [10]:
embedding = np.asarray(embedding)

In [11]:
attention_mask = np.asarray(attention_mask)

<b>6. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtAlbert model<b>

In [12]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [13]:
print(features)

[array([[ 0.05041175,  0.01670779, -0.06041724, ...,  0.03879363,
         0.06742962, -0.02244038],
       [ 0.04481451,  0.02007474, -0.06041677, ...,  0.01449978,
         0.07357263, -0.02993902],
       [ 0.0443356 ,  0.02113769, -0.05988102, ...,  0.01485755,
         0.07245725, -0.02932552],
       ...,
       [ 0.04479425,  0.01773441, -0.06211121, ...,  0.01833858,
         0.070872  , -0.02901525],
       [ 0.03966124,  0.04050408, -0.01892279, ...,  0.02172116,
         0.06361916, -0.04445174],
       [ 0.04467406,  0.02889616, -0.06304057, ...,  0.00830591,
         0.08290309, -0.02485517]], dtype=float32), array([[ 0.03748778,  0.06098351, -0.00593938, ...,  0.03400151,
        -0.08301088,  0.03961181],
       [ 0.01534716,  0.07612924,  0.00925653, ...,  0.02267499,
        -0.06700575,  0.04933011],
       [ 0.01369366,  0.07732251,  0.01009464, ...,  0.02235255,
        -0.064744  ,  0.059073  ],
       [ 0.01434124,  0.07683621,  0.00690954, ...,  0.02471154,
     